<a href="https://colab.research.google.com/github/Aftabgazali/Fashion-Product-Classification-Using-CNN/blob/main/Fashion_Product_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fash=keras.datasets.fashion_mnist

In [3]:
(train_images,train_label),(test_images,test_labels)=fash.load_data()

4423680/4422102 [==============================] - 0s 0us/step


## Data Preprocessing

In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images=train_images.reshape(60000,28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

# Hyperparameter Tuning

In [6]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=af91137aeeec554d53f41ab4d3f679d9e76f61327e3c44c65289d63985fb9dac
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=1edccb65869f4592dfc7377bbb26d1a6383c19df4754f3504df75daf82368018
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_label,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 33s]
val_accuracy: 0.8606666922569275

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 04m 01s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.fit(train_images, train_label, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1258 - accuracy: 0.9530 - val_loss: 0.2633 - val_accuracy: 0.9118
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0861 - accuracy: 0.9679 - val_loss: 0.3019 - val_accuracy: 0.9172
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0556 - accuracy: 0.9796 - val_loss: 0.3311 - val_accuracy: 0.9148
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0417 - accuracy: 0.9848 - val_loss: 0.4232 - val_accuracy: 0.9128
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0310 - accuracy: 0.9892 - val_loss: 0.4675 - val_accuracy: 0.9128
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0246 - accuracy: 0.9908 - val_loss: 0.4906 - val_accuracy: 0.9097
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0246 - accuracy: 0.9918 - val_loss: 0.5200 - val_a

In [15]:
model.evaluate(test_images,test_labels,batch_size=32)

313/313 [==============================] - 2s 5ms/step - loss: 0.5312 - accuracy: 0.9130


[0.5311602354049683, 0.9129999876022339]